In [2]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

chat_history = []

def print_logo():
    logo = [
        " ______ ___ ____     ______ __  __ ___   ______      ____   ____  ______",
        "/_  __/<  // __ \   / ____// / / //   | /_  __/     / __ ) / __ \/_  __/",
        " / /   / // / / /  / /    / /_/ // /| |  / /______ / __  |/ / / / / /",
        "/ /   / // /_/ /  / /___ / __  // ___ | / //_____// /_/ // /_/ / / /",
        "/_/   /_//_____/   \____//_/ /_//_/  |_|/_/       /_____/ \____/ /_/",
        "   ____ _ ___   ____  <  /",
        "  / __ `// _ \ / __ \ / /",
        " / /_/ //  __// / / // /",
        " \__, / \___//_/ /_//_/",
        "/____/                   "
    ]
    
    for line in logo:
        print(line)
    print()


def percentage_time_in_range(df, lower, upper):
    total_time = (df['Timestamp (YYYY-MM-DDThh:mm:ss)'].max() - df['Timestamp (YYYY-MM-DDThh:mm:ss)'].min()).total_seconds() / 3600
    time_in_range = df[(df['Glucose Value (mg/dL)'] >= lower) & (df['Glucose Value (mg/dL)'] <= upper)].shape[0] * 5 / 3600  # assuming each row is 5 minutes
    return (time_in_range / total_time) * 100

def analyze_glucose_data():
    file_name = input("Enter the name and path of the CSV file containing glucose data: (e.g., documents/chatbot/example.csv)")
    try:
        df = pd.read_csv(file_name)
        df['Timestamp (YYYY-MM-DDThh:mm:ss)'] = pd.to_datetime(df['Timestamp (YYYY-MM-DDThh:mm:ss)'])
        df['Glucose Value (mg/dL)'] = pd.to_numeric(df['Glucose Value (mg/dL)'], errors='coerce')
        df = df.dropna(subset=['Glucose Value (mg/dL)'])
        
        # Calculate average glucose value
        average_glucose = df['Glucose Value (mg/dL)'].mean()

        # Calculate time spent in each range
        good_range_percentage = percentage_time_in_range(df, 90, 140)

        # Find times of day associated with dips and spikes over 220 mg/dL
        spike_times = df[df['Glucose Value (mg/dL)'] > 220]['Timestamp (YYYY-MM-DDThh:mm:ss)'].dt.strftime('%I:%M %p').value_counts().idxmax()
        dip_times = df[df['Glucose Value (mg/dL)'] < 70]['Timestamp (YYYY-MM-DDThh:mm:ss)'].dt.strftime('%I:%M %p').value_counts().idxmax()

        # Plot
        plt.figure(figsize=(12, 6))
        plt.plot(df['Timestamp (YYYY-MM-DDThh:mm:ss)'], df['Glucose Value (mg/dL)'], color='black', label='Glucose Level')

        ranges = [(90, 140, 'green'), (70, 90, 'yellow'), (140, 220, 'yellow'), (float('-inf'), 70, 'red'), (220, float('inf'), 'red')]
        for lower, upper, color in ranges:
            plt.axhspan(lower, upper, color=color, alpha=0.3)

        plt.axhline(y=average_glucose, color='black', linestyle='--', label='Average Glucose')

        plt.title('Blood Sugar Over Time')
        plt.xlabel('Time')
        plt.ylabel('Glucose Value (mg/dL)')
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()
        chat_history.append(f"Average Glucose Value: {average_glucose}")
        chat_history.append(f"Percentage of time spent in good range (90-140 mg/dL): {round(good_range_percentage, 2)} %")
        chat_history.append(f"Most common time for spikes over 220 mg/dL: {spike_times}")
        chat_history.append(f"Most common time for dips below 70 mg/dL: {dip_times}")
        print("Average Glucose Value:", average_glucose)
        print("Percentage of time spent in good range (90-140 mg/dL):", round(good_range_percentage, 2), "%")
        print("Most common time for spikes over 220 mg/dL:", spike_times)
        print("Most common time for dips below 70 mg/dL:", dip_times)
        print("Chatbot: Here's the analysis of your glucose data.")
    except FileNotFoundError:
        print("File not found. Please make sure the file name is correct.")
# database connection
conn = sqlite3.connect('food_nutrition.db')
c = conn.cursor()

def setup_database():
    c.execute('''CREATE TABLE IF NOT EXISTS FoodNutrition (
                    FoodName TEXT PRIMARY KEY,
                    Weight TEXT,
                    Kcals REAL,
                    Protein REAL,
                    Fats REAL,
                    Carbs REAL,
                    Fibre REAL
                )''')

    # Sample data
    sample_data = [
        ('apple', '100g', 52, 0.3, 0.2, 14, 2.4),
        ('banana', '100g', 89, 1.1, 0.3, 23, 2.6),
        ('chicken breast', '100g', 165, 31, 3.6, 0, 0),
        ('white rice', '100g', 130, 2.7, 0.3, 28, 0.4),
        ('whole wheat bread', '100g', 247, 9.4, 3.3, 47, 6.1),
        ('spinach', '100g', 23, 2.9, 0.4, 3.6, 2.2),
        ('salmon', '100g', 208, 20, 13, 0, 0),
        ('yogurt', '100g', 61, 3.5, 3.3, 4.7, 0),
        ('quinoa', '100g', 120, 4.4, 1.9, 21, 2.8),
        ('avocado', '100g', 160, 2, 15, 9, 7),
        ('almonds', '100g', 579, 21, 49, 22, 12),
        ('broccoli', '100g', 34, 2.8, 0.4, 6.6, 2.6),
        ('oatmeal', '100g', 68, 2.5, 1.4, 12, 1.7),
        ('carrots', '100g', 41, 0.9, 0.2, 10, 2.8),
        ('sweet potato', '100g', 86, 1.6, 0.1, 20, 3),
        ('salad greens', '100g', 15, 1.4, 0.2, 2.9, 1.5),
        ('tomato', '100g', 18, 0.9, 0.2, 3.9, 1.2),
        ('blueberries', '100g', 57, 0.7, 0.3, 14, 2.4),
        ('black beans', '100g', 339, 21, 1.4, 63, 16),
        ('chickpeas', '100g', 164, 8.9, 2.6, 27, 7.6),
        ('eggs', '100g', 155, 13, 11, 1.1, 0),
        ('cheese', '100g', 402, 25, 33, 1.3, 0),
        ('milk', '100g', 42, 3.4, 1, 4.8, 0),
        ('tuna', '100g', 116, 26, 1, 0, 0),
        ('ground beef', '100g', 250, 27, 17, 0, 0),
        ('pasta', '100g', 131, 5.8, 0.6, 25, 1.3),
        ('potato', '100g', 77, 2, 0.1, 17, 2.2),
        ('peanut butter', '100g', 589, 25, 50, 16, 8),
        ('honey', '100g', 304, 0.3, 0, 82, 0.2),
        ('olive oil', '100g', 884, 0, 100, 0, 0),
        ('chocolate', '100g', 546, 5.5, 31, 61, 7),
        ('orange', '100g', 47, 0.9, 0.1, 12, 2.4),
        ('peach', '100g', 39, 0.9, 0.2, 10, 1.5),
        ('pear', '100g', 57, 0.4, 0.1, 15, 3.1),
        ('grapes', '100g', 69, 0.7, 0.2, 18, 0.9),
        ('kiwi', '100g', 61, 1.1, 0.5, 15, 3),
        ('melon', '100g', 34, 0.8, 0.2, 8.2, 0.9),
        ('strawberries', '100g', 32, 0.7, 0.3, 8.3, 2),
        ('watermelon', '100g', 30, 0.6, 0.2, 8, 0.4),
        ('peanut', '100g', 567, 25, 49, 16, 8),
        ('walnuts', '100g', 654, 15, 65, 14, 7),
        ('cashews', '100g', 553, 18, 44, 30, 3.3),
        ('pecans', '100g', 691, 9.2, 72, 14, 9.6),
        ('pine nuts', '100g', 673, 13, 68, 13, 4.3),
        ('sunflower seeds', '100g', 584, 20, 51, 20, 8.6),
        ('pistachios', '100g', 562, 20, 45, 28, 10),
        ('soybeans', '100g', 173, 16, 9, 10, 6),
        ('corn', '100g', 86, 3.2, 1.2, 19, 2.7),
        ('peas', '100g', 81, 5, 0.4, 14, 5),
        ('dark chocolate', '100g', 598, 7.8, 42, 51, 7),
        ('milk chocolate', '100g', 535, 7.3, 30, 60, 3.4),
        ('vanilla ice cream', '100g', 207, 3.5, 11, 24, 0.7),
        ('french fries', '100g', 312, 3.4, 15, 42, 3.4),
        ('pizza', '100g', 267, 10, 10, 31, 1.4),
        ('hamburger', '100g', 264, 12, 16, 17, 0),
        ('hot dog', '100g', 296, 12, 24, 2.5, 0),
        ('fried chicken', '100g', 312, 24, 21, 0, 0),
        ('ice cream', '100g', 207, 3.5, 11, 24, 0.7),
        ('chips', '100g', 536, 6.7, 33, 53, 4.1),
        ('cake', '100g', 237, 3.4, 12, 31, 0.5),
        ('donut', '100g', 452, 5.9, 25, 51, 1.4),
        ('cookies', '100g', 486, 5.4, 24, 64, 1.7),
        ('brownies', '100g', 466, 4.2, 23, 62, 2.4)
    ]
    # Insert sample data
    c.executemany('INSERT OR IGNORE INTO FoodNutrition VALUES (?, ?, ?, ?, ?, ?, ?)', sample_data)

    conn.commit()

def respond_to_input(input_number):
    if input_number.lower() == 'quit':  # Check for quit command first
        return None
    elif input_number == '1':
        return "If you are experiencing low blood sugar, take 15 grams of sugars and wait 15 minutes."
    elif input_number == '2':
        return "If you are experiencing high blood sugar, a general guideline is to take 1 unit of insulin for every 50 mg above range. Consider drinking water or taking a walk to help lower sugars."
    elif input_number == '3':
        response = calculate_insulin_dose()
        return response
    elif input_number == '4':
        food_name = input("Enter the food you want nutritional information for: ")
        food_info = get_nutritional_info(food_name)
        if food_info:
            response = format_nutritional_info(food_name, food_info)
            return response
        else:
            return "Sorry, nutritional information for that food is not available."
    elif input_number == '5':
        response = "In case of diabetes-related emergencies, please call the National Diabetes Emergency Hotline at 1-800-XXX-XXXX for immediate assistance."
        return response
    elif input_number == '6':
        analyze_glucose_data()
        chat_history.append("Chatbot: Here's the analysis of your glucose data.")
        return None
    else:
        return "I'm sorry, I didn't understand. Could you please repeat or ask a different question?"
def calculate_insulin_dose():
    insulin_to_carb_ratio = float(input("Enter your insulin to carb ratio: "))
    carbs_in_meal = float(input("Enter the amount of carbs in your meal (grams): "))
    blood_sugar = float(input("Enter your current blood sugar level (mg/dL): "))
    
    # Calculate insulin dose based on carb intake
    insulin_dose = carbs_in_meal / insulin_to_carb_ratio
    
    # Adjust insulin dose based for blood sugar level
    if blood_sugar > 100:
        # For every 50 mg above 100 mg/dL, add 1 unit of insulin
        insulin_correction = (blood_sugar - 100) / 50
        insulin_dose += insulin_correction
    
    elif blood_sugar < 100:
        # For every 50 mg below 100 mg/dL, subtract 1 unit of insulin
        insulin_correction = (100 - blood_sugar) / 50
        insulin_dose -= insulin_correction
    
    return f"Your insulin dose for {carbs_in_meal} grams of carbs and blood sugar level {blood_sugar} mg/dL is {insulin_dose} units."

def get_nutritional_info(food_name):
    c.execute('SELECT * FROM FoodNutrition WHERE FoodName = ?', (food_name,))
    return c.fetchone()

def format_nutritional_info(food_name, food_info):
    return f"Nutritional information for {food_name}:\nWeight: {food_info[1]}\nKcals: {food_info[2]}\nProtein: {food_info[3]}\nFats: {food_info[4]}\nCarbs: {food_info[5]}\nFibre: {food_info[6]}"

def save_chat_history(chat_history, file_name):
    with open(file_name, 'w') as file:
        for entry in chat_history:
            file.write(entry + '\n')

def main():
    print_logo()
    setup_database()
    print("Welcome to the Diabetes Chatbot!")
    print("You can ask questions related to:\n1. Low blood sugar\n2. High blood sugar\n3. Insulin dosing\n4. Nutrition/food information\n5. Emergency\n6. Analyze Glucose Levels")
    print("Type 'quit' to exit.")

    df = None
    average_glucose = None

    while True:
        user_input = input("You: ")
        chat_history.append(f"You: {user_input}")  # Record user input
        if user_input.lower() == 'quit':
            print("Goodbye! Don't forget to save the Graph by right clicking and using 'Save As'")
            chat_history.append("User quit the conversation.")
            save_chat_history(chat_history, input("Chat history will be saved as: ") + ".txt")
            print('Chat history saved successfully')
            break
        elif user_input.isdigit() and 0 <= int(user_input) <= 6:
            if user_input == '0':
                menu = "You can ask questions related to:\n1. Low blood sugar\n2. High blood sugar\n3. Insulin dosing\n4. Nutrition/food information\n5. Emergency\n6. Analyze Glucose Levels"
                print(menu)
                chat_history.append(menu)
            else:
                response = respond_to_input(user_input)
                chat_history.append(f"Chatbot: {response}")
                if response is not None:
                    print("\nChatbot:")
                    print("-" * 30)
                    print(response)
                    print("-" * 30 + "\n")

                # Prompt user to enter a choice again
                print("Type 'quit' to exit or enter 0 to see the menu again")
        else:
            print("Please enter a number from 0 to 6 or 'quit' to exit.")

main()

 ______ ___ ____     ______ __  __ ___   ______      ____   ____  ______
/_  __/<  // __ \   / ____// / / //   | /_  __/     / __ ) / __ \/_  __/
 / /   / // / / /  / /    / /_/ // /| |  / /______ / __  |/ / / / / /
/ /   / // /_/ /  / /___ / __  // ___ | / //_____// /_/ // /_/ / / /
/_/   /_//_____/   \____//_/ /_//_/  |_|/_/       /_____/ \____/ /_/
   ____ _ ___   ____  <  /
  / __ `// _ \ / __ \ / /
 / /_/ //  __// / / // /
 \__, / \___//_/ /_//_/
/____/                   

Welcome to the Diabetes Chatbot!
You can ask questions related to:
1. Low blood sugar
2. High blood sugar
3. Insulin dosing
4. Nutrition/food information
5. Emergency
6. Analyze Glucose Levels
Type 'quit' to exit.


KeyboardInterrupt: Interrupted by user